In [1]:
import pandas as pd

# 讀取資料集
df = pd.read_csv('customer_churn.csv')

In [2]:
#第1題

from sklearn.model_selection import StratifiedShuffleSplit

#取出60%
split = StratifiedShuffleSplit(n_splits=1, train_size=0.6, random_state=15)

for train_index, test_index in split.split(df, df['Churn']):
    new_set = df.loc[train_index]
    test_set = df.loc[test_index]


# 檢查分割結果
new_set_size = len(new_set)
test_set_size = len(test_set)

new_set_ratio = new_set_size / (new_set_size + test_set_size)
test_set_ratio = test_set_size / (new_set_size + test_set_size)

print("New set size: ", new_set_size, ", ratio: {:.2%}".format(new_set_ratio))
#print("Test set size: ", test_set_size, ", ratio: {:.2%}".format(test_set_ratio))

print("")

origin_churn_percent = df['Churn'].value_counts(normalize=True)
print("Origin set churn percentage:\n", origin_churn_percent)

new_churn_percent = new_set['Churn'].value_counts(normalize=True)
print("New set churn percentage:\n", new_churn_percent)

#test_churn_percent = test_set['Churn'].value_counts(normalize=True)
#print("Test set churn percentage:\n", test_churn_percent)

New set size:  3384 , ratio: 60.00%

Origin set churn percentage:
 0    0.831383
1    0.168617
Name: Churn, dtype: float64
New set churn percentage:
 0    0.831265
1    0.168735
Name: Churn, dtype: float64


In [3]:
#第2題
print(new_set['Churn'].value_counts())

0    2813
1     571
Name: Churn, dtype: int64


In [4]:
#第3題
print("\n每個欄位的空值個數：")
print(new_set.isnull().sum())


每個欄位的空值個數：
CustomerID                       0
Churn                            0
Tenure                         154
PreferredLoginDevice             0
CityTier                         0
WarehouseToHome                141
PreferredPaymentMode             0
Gender                           0
HourSpendOnApp                 148
NumberOfDeviceRegistered         0
PreferedOrderCat                 0
SatisfactionScore                0
MaritalStatus                    0
NumberOfAddress                  0
Complain                         0
OrderAmountHikeFromlastYear    157
CouponUsed                     140
OrderCount                     160
DaySinceLastOrder              189
CashbackAmount                   0
dtype: int64


In [5]:
#第3題-續
from sklearn.preprocessing import LabelEncoder

# 使用平均數填補空值
new_set['Tenure'].fillna(new_set['Tenure'].mean(), inplace=True)
new_set['WarehouseToHome'].fillna(new_set['WarehouseToHome'].mean(), inplace=True)
new_set['HourSpendOnApp'].fillna(new_set['HourSpendOnApp'].mean(), inplace=True)
new_set['OrderAmountHikeFromlastYear'].fillna(new_set['OrderAmountHikeFromlastYear'].mean(), inplace=True)
new_set['CouponUsed'].fillna(new_set['CouponUsed'].mean(), inplace=True)
new_set['OrderCount'].fillna(new_set['OrderCount'].mean(), inplace=True)
new_set['DaySinceLastOrder'].fillna(new_set['DaySinceLastOrder'].mean(), inplace=True)

#把nominal欄位轉換為numeric型態
le = LabelEncoder()
new_set = pd.get_dummies(new_set, columns=['PreferredLoginDevice'])
new_set = pd.get_dummies(new_set, columns=['PreferredPaymentMode'])
new_set['Gender'] = le.fit_transform(new_set['Gender'])
new_set = pd.get_dummies(new_set, columns=['PreferedOrderCat'])
new_set = pd.get_dummies(new_set, columns=['MaritalStatus'])

#把預測欄位轉為nominal
new_set['Churn'] = new_set['Churn'].astype('category')

#刪掉id欄
new_set = new_set.drop('CustomerID', axis=1)

In [6]:
#第4題
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
import numpy as np

X = new_set.drop("Churn", axis=1)
y = new_set["Churn"]

# 使用 Logistic Regression 建立模型並評估
lr = LogisticRegression(penalty='l1', C=1, solver='liblinear', max_iter=1000)
lr_scores = cross_val_score(lr, X, y, cv=10,scoring='accuracy')
# print("Logistic Regression scores:")
# print(np.array2string(lr_scores, separator='\n'))
# print("Mean score:", lr_scores.mean())



# 使用 SVM 建立模型並評估
svm = SVC(kernel='rbf', C=5, gamma='auto')
svm_scores = cross_val_score(svm, X, y, cv=10,scoring='accuracy')
# print("SVM scores:")
# print(np.array2string(svm_scores, separator='\n'))
# print("Mean score:", svm_scores.mean())


In [16]:
#第5題

# 輸出每個模型的交叉驗證分數
print('Logistic Regression 10-fold cross-validation scores:', lr_scores)
print('SVM 10-fold cross-validation scores:', svm_scores)

Logistic Regression 10-fold cross-validation scores: [0.87610619 0.90855457 0.89380531 0.89380531 0.8964497  0.89349112
 0.8964497  0.84911243 0.89349112 0.90828402]
SVM 10-fold cross-validation scores: [0.92625369 0.92035398 0.92035398 0.90560472 0.89053254 0.91715976
 0.9260355  0.93786982 0.91420118 0.93491124]


In [26]:
# # 對測試集進行預測
# lr_pred = lr.predict(test_set.drop('Churn', axis=1))
# svm_pred = svm.predict(test_set.drop('Churn', axis=1))

# # 計算模型的準確率
# lr_acc = accuracy_score(test_set['Churn'], lr_pred)
# svm_acc = accuracy_score(test_set['Churn'], svm_pred)

# # 印出結果
# print("Logistic Regression accuracy:", lr_acc)
# print("SVM accuracy:", svm_acc)

NotFittedError: This LogisticRegression instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [18]:

from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
import numpy as np

all_lr_scores = []
all_svm_scores = []

for i in range(30):
    # 假設你的 dataframe 為 df，Churn 欄位名稱為 'Churn'
    split = StratifiedShuffleSplit(n_splits=1, train_size=0.6, random_state=15)

    for train_index, test_index in split.split(df, df['Churn']):
        new_set = df.loc[train_index]
        test_set = df.loc[test_index]

    # 使用平均數填補空值
    new_set['Tenure'].fillna(new_set['Tenure'].mean(), inplace=True)
    new_set['WarehouseToHome'].fillna(new_set['WarehouseToHome'].mean(), inplace=True)
    new_set['HourSpendOnApp'].fillna(new_set['HourSpendOnApp'].mean(), inplace=True)
    new_set['OrderAmountHikeFromlastYear'].fillna(new_set['OrderAmountHikeFromlastYear'].mean(), inplace=True)
    new_set['CouponUsed'].fillna(new_set['CouponUsed'].mean(), inplace=True)
    new_set['OrderCount'].fillna(new_set['OrderCount'].mean(), inplace=True)
    new_set['DaySinceLastOrder'].fillna(new_set['DaySinceLastOrder'].mean(), inplace=True)

    #把nominal欄位轉換為numeric型態
    le = LabelEncoder()
    new_set = pd.get_dummies(new_set, columns=['PreferredLoginDevice'])
    new_set = pd.get_dummies(new_set, columns=['PreferredPaymentMode'])
    new_set['Gender'] = le.fit_transform(new_set['Gender'])
    new_set = pd.get_dummies(new_set, columns=['PreferedOrderCat'])
    new_set = pd.get_dummies(new_set, columns=['MaritalStatus'])

    #把預測欄位轉為nominal
    new_set['Churn'] = new_set['Churn'].astype('category')
    
    #刪掉id欄
    new_set = new_set.drop('CustomerID', axis=1)

    X = new_set.drop("Churn", axis=1)
    y = new_set["Churn"]

    # 使用 Logistic Regression 建立模型並評估
    lr = LogisticRegression(penalty='l1', C=1, solver='liblinear', max_iter=1000)
    lr_scores = cross_val_score(lr, X, y, cv=10,scoring='accuracy')
    all_lr_scores.append(lr_scores.mean())

    # 使用 SVM 建立模型並評估
    svm = SVC(kernel='rbf', C=5, gamma='auto')
    svm_scores = cross_val_score(svm, X, y, cv=10,scoring='accuracy')
    all_svm_scores.append(svm_scores.mean())

lr_average = sum(all_lr_scores) / len(all_lr_scores)
svm_average = sum(all_svm_scores) / len(all_svm_scores)
print("lr Average:", lr_average)
print("svm Average:", svm_average)

lr Average: 0.8909648112268946
svm Average: 0.919327643085302


In [20]:
#第7題
import scipy.stats as stats

lr_mean = np.mean(all_lr_scores)
svm_mean = np.mean(all_svm_scores)

diff_mean = lr_mean - svm_mean

lr_std = np.std(all_lr_scores, ddof=1)
svm_std = np.std(all_svm_scores, ddof=1)

n = len(all_lr_scores)
df = n - 1

std_err = np.sqrt((lr_std**2 + svm_std**2) / n)
t = diff_mean / std_err
p = stats.t.sf(np.abs(t), df)*2

print("LR mean:", lr_mean)
print("SVM mean:", svm_mean)
print("Difference in means:", diff_mean)

print("LR std:", lr_std)
print("SVM std:", svm_std)

print("Std err:", std_err)
print("t-value:", t)
print("p-value:", p)

LR mean: 0.8909648112268944
SVM mean: 0.9193276430853018
Difference in means: -0.028362831858407378
LR std: 5.401603131214479e-05
SVM std: 3.387607712450221e-16
Std err: 9.861932938855693e-06
t-value: -2875.9911504426022
p-value: 1.166564276041105e-80


由於p-value極小，所以這兩個模型在統計上不存在顯著差異